# Tableau uploads

## Imports

In [1]:
import pandas as pd
import numpy as np
import wrangle as w
import explore as e

In [2]:
df = w.clean_311(w.get_311_data())

In [3]:
df.head()

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,...,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",...,0,0,0.086,67656,78203,1,2017,4,19055,59.81
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",...,0,0,0.078,66370,78242,1,2017,4,18500,65.21
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",...,0,0,0.086,67656,78202,2,2017,5,19055,59.81
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",...,0,0,0.148,68081,78207,2,2017,5,23967,26.00
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",...,0,0,0.124,80007,78251,2,2017,9,23437,38.44


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356804 entries, 551 to 399979
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   open_date                 356804 non-null  datetime64[ns]
 1   due_date                  356804 non-null  datetime64[ns]
 2   closed_date               356804 non-null  datetime64[ns]
 3   is_late                   356804 non-null  object        
 4   dept                      356804 non-null  object        
 5   call_reason               356804 non-null  object        
 6   case_type                 356804 non-null  object        
 7   case_status               356804 non-null  object        
 8   source_id                 356804 non-null  object        
 9   address                   356804 non-null  object        
 10  council_district          356804 non-null  int64         
 11  longitude                 356804 non-null  float64       
 12  

In [5]:
df.level_of_delay.value_counts()

Very Early Response    275450
Late Response           30415
Early Response          24303
On Time Response        21001
Very Late Response       5635
Name: level_of_delay, dtype: int64

In [6]:
df.is_late.value_counts()

NO     305729
YES     51075
Name: is_late, dtype: int64

In [7]:
df.days_before_or_after_due.value_counts()

 2.0       35087
 5.0       28539
 4.0       27069
 6.0       26922
 1.0       26317
           ...  
-1247.0        1
 434.0         1
-765.0         1
-541.0         1
 532.0         1
Name: days_before_or_after_due, Length: 1319, dtype: int64

In [8]:
on_time = df[df.level_of_delay == 'On Time Response']
very_late = df[df.level_of_delay == 'Very Late Response']
early = df[df.level_of_delay == 'Early Response']

In [9]:
on_time.days_before_or_after_due.value_counts()

 0.0      8780
-1.0      7060
 1.0      1904
 2.0       363
 14.0      255
 13.0      232
 8.0       223
 3.0       219
 7.0       216
 15.0      208
 5.0       206
 6.0       195
 4.0       143
 12.0      142
 11.0      140
 9.0       133
 10.0      111
 17.0       43
 18.0       37
 20.0       35
 16.0       32
 21.0       31
 19.0       29
 27.0       28
 29.0       25
 23.0       25
 28.0       23
 22.0       22
 25.0       20
 30.0       15
 26.0       14
 24.0       11
 31.0       11
 40.0       10
 34.0        9
 32.0        8
 35.0        7
 33.0        7
 39.0        5
 41.0        4
 43.0        4
 36.0        4
 37.0        3
 42.0        3
 44.0        2
 38.0        2
 132.0       1
 129.0       1
Name: days_before_or_after_due, dtype: int64

In [10]:
very_late.days_before_or_after_due.value_counts()

-109.0     110
-20.0       81
-108.0      69
-107.0      69
-110.0      66
          ... 
-226.0       1
-760.0       1
-1254.0      1
-333.0       1
-1344.0      1
Name: days_before_or_after_due, Length: 936, dtype: int64

In [11]:
early.days_before_or_after_due.value_counts()

1.0      6637
2.0      5275
0.0      1986
3.0      1799
4.0       944
28.0      520
27.0      444
29.0      431
30.0      399
5.0       378
22.0      371
26.0      355
6.0       348
21.0      312
20.0      312
31.0      300
23.0      285
25.0      269
24.0      246
19.0      194
32.0      174
17.0      148
16.0      135
7.0       132
13.0       97
18.0       95
12.0       80
35.0       77
33.0       75
14.0       74
34.0       73
11.0       67
10.0       63
36.0       56
57.0       50
37.0       46
56.0       43
8.0        42
62.0       41
9.0        41
47.0       38
42.0       38
49.0       35
43.0       33
54.0       32
41.0       31
50.0       30
38.0       30
63.0       29
55.0       28
48.0       28
58.0       27
15.0       27
39.0       26
52.0       25
53.0       25
44.0       24
64.0       24
46.0       23
51.0       22
61.0       21
45.0       20
40.0       19
60.0       18
59.0       17
77.0       16
78.0       15
69.0       14
76.0       12
70.0       12
79.0       11
80.0  

In [12]:
df[df.days_before_or_after_due < 0]

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,...,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",...,0,0,0.086,67656,78203,1,2017,4,19055,59.81
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",...,0,0,0.078,66370,78242,1,2017,4,18500,65.21
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",...,0,0,0.086,67656,78202,2,2017,5,19055,59.81
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",...,0,0,0.148,68081,78207,2,2017,5,23967,26.00
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",...,0,0,0.124,80007,78251,2,2017,9,23437,38.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395820,2021-05-12,2021-05-12,2021-05-13,YES,Animal Care Services,field,Animals(Aggressive Critical),Closed,Web Portal,"407 MAYBERRY AVE, SAN ANTONIO, 78228",...,0,0,0.085,61418,78228,5,2021,19,13836,22.24
396290,2021-05-12,2021-05-13,2021-05-14,YES,Animal Care Services,field,Trapped/Confined,Closed,Web Portal,"3702 W POPLAR ST, SAN ANTONIO, 78228",...,0,0,0.085,61418,78228,5,2021,19,13836,22.24
396597,2021-05-12,2021-05-13,2021-05-14,YES,Animal Care Services,field,Trapped/Confined,Closed,Web Portal,"8303 GREATVIEW, SAN ANTONIO, 78230",...,0,0,0.137,97717,78230,5,2021,19,35475,71.64
396659,2021-05-12,2021-05-13,2021-05-14,YES,Animal Care Services,field,Trapped/Confined,Closed,Web Portal,"17003 TALON PATH, SAN ANTONIO, 78247",...,0,1,0.154,91378,78247,5,2021,19,30240,55.62


In [13]:
def get_late(days_before_or_after_due):
    if days_before_or_after_due < 0:
        return 'late'
    else:
        return 'on time'
        

In [14]:
# create new feature to show how long it took to resolve compared to resolution due date
#df['days_before_or_after_due'] = df.resolution_days_due - df.days_open

In [33]:
import model as m

In [34]:
df = m.model_df()

In [41]:
  
import pandas as pd
import numpy as np
import wrangle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing

#-----------------------------------------------------------------------------

# Model Prep

def dummy_dept(df):
    # dummy dept feature
    dummy_df =  pd.get_dummies(df['dept'])
    # Name the new columns
    dummy_df.columns = ['animal_care_services', 'code_enforcement_services', 
                        'customer_services', 'development_services', 
                        'metro_health', 'parks_and_rec',
                        'solid_waste_management', 'trans_and_cap_improvements', 
                        'unknown_dept']
    # add the dummies to the data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df
#-----------------------------------------------------------------------------    
def dummy_call_reason(df):
    # dummy dept feature
    dummy_df =  pd.get_dummies(df['call_reason'])
    # Name the new columns
    dummy_df.columns = ['buildings', 'business', 'cleanup', 'code',
                        'customer_service', 'field', 'land',
                        'license', 'misc', 'storm', 'streets', 'trades', 
                        'traffic', 'waste']
    # add the dummies to the data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df
#-----------------------------------------------------------------------------
def make_source_id_dummies(df):
    '''This function takes in the cleaned dataframe, makes dummy variables of the source id column, readds the names of the
    dummy columns and returns the concatenated dummy dataframe to the original dataframe.'''
    #make dummies
    dummy_df = pd.get_dummies(df['source_id'])
    #add back column names
    dummy_df.columns = ['web_portal', '311_mobile_app', 'constituent_call', 'internal_services_requests']
    # concatenate dummies to the cleaned data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df

#-------------------------------
def keep_info(df):
    df.drop(df.columns.difference(['dept','call_reason', 'source_id', 'level_of_delay',
                                   'council_district', 'resolution_days_due', 'district_0', 'district_1', 'district_2',
                                   'district_3', 'district_4','district_5', 'district_6', 'district_7', 'district_8', 
                                   'district_9','district_10']), 1, inplace=True)
    return df

#--------------------------------
def model_df():
    '''This function reads in the clean 311 dataframe, applies all of the above functions to prepare it for modeling. 
    The function then returns a cleaned dataframe ready for modeling.'''
    df= wrangle.clean_311(wrangle.get_311_data())
    df= keep_info(df)
    df= dummy_dept(df)
    df= dummy_call_reason(df)
    df= make_source_id_dummies(df)

    return df
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def split(df, stratify_by= 'level_of_delay'):
    """
    Crude train, validate, test split
    To stratify, send in a column name
    """
    if stratify_by == None:
        train, test = train_test_split(df, test_size=.2, random_state=319)
        train, validate = train_test_split(train, test_size=.3, random_state=319)
    else:
        train, test = train_test_split(df, test_size=.2, random_state=319, stratify=df[stratify_by])
        train, validate = train_test_split(train, test_size=.3, random_state=319, stratify=train[stratify_by])
    return train, validate, test


#------------------------------------
def separate_y(train, validate, test):
    '''
    This function will take the train, validate, and test dataframes and separate the target variable into its
    own panda series
    '''
    
    X_train = train.drop(columns=['level_of_delay'])
    y_train = train.level_of_delay
    X_validate = validate.drop(columns=['level_of_delay'])
    y_validate = validate.level_of_delay
    X_test = test.drop(columns=['level_of_delay'])
    y_test = test.level_of_delay
    return X_train, y_train, X_validate, y_validate, X_test, y_test
#------------------------------------
def scale_data(X_train, X_validate, X_test):
    '''
    This function will scale numeric data using Min Max transform after 
    it has already been split into train, validate, and test.
    '''
    
    
    obj_col = ['dept', 'call_reason','source_id']
    num_train = X_train.drop(columns = obj_col)
    num_validate = X_validate.drop(columns = obj_col)
    num_test = X_test.drop(columns = obj_col)
    
    
    # Make the thing
    scaler = preprocessing.MinMaxScaler()
    
   
    # we only .fit on the training data
    scaler.fit(num_train)
    train_scaled = scaler.transform(num_train)
    validate_scaled = scaler.transform(num_validate)
    test_scaled = scaler.transform(num_test)
    
    # turn the numpy arrays into dataframes
    train_scaled = pd.DataFrame(train_scaled, columns=num_train.columns)
    validate_scaled = pd.DataFrame(validate_scaled, columns=num_train.columns)
    test_scaled = pd.DataFrame(test_scaled, columns=num_train.columns)
    
    
    return train_scaled, validate_scaled, test_scaled

#------------------------------------

def split_separate_scale(df, stratify_by= 'level_of_delay'):
    '''
    This function will take in a dataframe
    separate the dataframe into train, validate, and test dataframes
    separate the target variable from train, validate and test
    then it will scale the numeric variables in train, validate, and test
    finally it will return all dataframes individually
    '''
    
    # split data into train, validate, test
    train, validate, test = split(df, stratify_by= 'level_of_delay')
    
     # seperate target variable
    X_train, y_train, X_validate, y_validate, X_test, y_test = separate_y(train, validate, test)
    
    
    # scale numeric variable
    train_scaled, validate_scaled, test_scaled = scale_data(X_train, X_validate, X_test)
    
    return train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled

In [42]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = split_separate_scale(df, stratify_by= 'level_of_delay')

In [49]:
train.head()

,dept,call_reason,source_id,council_district,resolution_days_due,level_of_delay,district_1,district_2,district_3,district_4,...,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests
71261,Solid Waste Management,misc,Web Portal,2,3,Very Early Response,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
190425,Solid Waste Management,waste,Web Portal,2,8,Very Early Response,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
52722,Development Services,code,311 Mobile App,5,131,Very Early Response,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
268361,Solid Waste Management,waste,Web Portal,4,9,Very Early Response,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
54866,Development Services,code,Web Portal,5,66,Very Early Response,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [52]:
train = train.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district'])

In [58]:
train = train.drop(columns = 'level_of_delay')

In [15]:
#get actual amount of late cases
df['late'] = df['days_before_or_after_due'].apply(get_late)

In [16]:
df.head()

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,...,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles,late
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",...,0,0.086,67656,78203,1,2017,4,19055,59.81,late
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",...,0,0.078,66370,78242,1,2017,4,18500,65.21,late
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",...,0,0.086,67656,78202,2,2017,5,19055,59.81,late
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",...,0,0.148,68081,78207,2,2017,5,23967,26.00,late
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",...,0,0.124,80007,78251,2,2017,9,23437,38.44,late


In [17]:
df.late.value_counts()

on time    313694
late        43110
Name: late, dtype: int64

### Level of Delay by District

https://public.tableau.com/app/profile/lori.segovia/viz/LevelofDelaybyDistrict/District1

In [18]:
df.to_csv('all_311.csv')

In [59]:
# 1 and 5 downtown/central
central = train[(train.district_1 == 1) | (train.district_5 == 1)]
# 8 and 9
north = train[(train.district_8 == 1) | (train.district_9 == 1)]
# 4 and 3
south = train[(train.district_4 == 1) | (train.district_3 == 1)]
# 10 and 2
east = train[(train.district_10 == 1) | (train.district_2 == 1)]
#6 and 7
west = train[(train.district_6 == 1) | (train.district_7 == 1)]

In [44]:
#north = df[(df.council_district == 8) | (df.council_district == 9)]

In [45]:
#north.head()

In [46]:
north.council_district.value_counts()

9    10902
8    10835
Name: council_district, dtype: int64

In [23]:
avg_days_north = north.days_before_or_after_due.mean()

In [24]:
avg_days_south = south.days_before_or_after_due.mean()

In [25]:
avg_days_east = east.days_before_or_after_due.mean()

In [26]:
avg_days_west = west.days_before_or_after_due.mean()

In [27]:
avg_days_central = central.days_before_or_after_due.mean()

In [30]:
import scipy.stats as stats

In [ ]:
null = 'there is no difference in the means'
alt = 'there is a difference in the means'

In [62]:
t, p = stats.ttest_ind(north, south, equal_var=False)
t, p

(array([-1.04995329e+01,             nan,             nan, -2.43716273e+02,
        -1.85408218e+02,             nan,             nan,             nan,
         1.46977610e+02,  1.47886470e+02,             nan, -2.44346817e+01,
        -7.31582472e+00, -3.95222039e-01, -1.65141751e+01,  7.28104336e+00,
         1.00000000e+00,  3.08840439e+01, -1.28419657e+00, -9.26673318e-01,
         2.28720540e-01,  8.08411755e-01,  2.19081680e+00, -1.68117044e+01,
         5.46486576e-01, -2.42312447e+01, -2.39635695e+00,  8.63716137e+00,
        -4.91929213e+00,  5.19433442e+00, -4.12312993e+00, -5.84138643e+00,
         3.81278196e+00,  3.25937562e+01, -1.59302101e+01,  9.06597561e+00,
        -7.49624363e+00,  4.72555404e+00]),
 array([9.31750936e-026,             nan,             nan, 0.00000000e+000,
        0.00000000e+000,             nan,             nan,             nan,
        0.00000000e+000, 0.00000000e+000,             nan, 4.20388340e-131,
        2.58728849e-013, 6.92680962e-001, 4.